# Fantasy Analysis for Week 8

In [71]:
from sleeper_wrapper import League
from sleeper_wrapper import Players
import statistics
import plotly.graph_objects as go
import plotly.express as px
import requests
from plotly.subplots import make_subplots

LEAGUE_ID = "649912836461539328"
CURRENT_WEEK = 8
FORMAT = "half_ppr"
FANTASY_POSITIONS = ["QB", "RB", "WR", "TE", "FLEX", "DEF", "K"]
league = League(LEAGUE_ID)
sleeper_players = Players()

users = league.get_users()
rosters = league.get_rosters()

def pair_user_rosters(rosters):
    """pairs a users roster id with their user id"""
    pairings = {}

    for roster in rosters:
        pairings[roster["roster_id"]] = roster["owner_id"]

    return pairings


def pair_usernames(users):
    pairings = {}

    # this is stored dumb, but basically team name can be in metadata or it can
    # be outside of it. If a team has no name, we can just use their display name
    for user in users:
        try:
            pairings[user["user_id"]] = user["metadata"]["team_name"]
        except KeyError:  # stored dumb
            try:
                pairings[user["user_id"]] = user["team_name"]
            except KeyError:  # no team name
                pairings[user["user_id"]] = user["display_name"]

    return pairings


ROSTER_PAIRINGS = pair_user_rosters(rosters)
NAME_PAIRINGS = pair_usernames(users)
all_games = {}


def pair_matchups(games):
    """pair matchup data with users and their opponent"""
    scoreboards = {}

    for player in games:
        if player["matchup_id"] in scoreboards.keys():
            scoreboards[player["matchup_id"]
                        ][ROSTER_PAIRINGS[player["roster_id"]]] = player
        else:
            scoreboards[player["matchup_id"]] = {
                ROSTER_PAIRINGS[player["roster_id"]]: player}

    return scoreboards


# creates a dictionary with all matchups in the league through CURRENT_WEEK
for week in range(1, CURRENT_WEEK+1):
    all_games[week] = pair_matchups(league.get_matchups(week))


In [7]:
# how does median points scored differ from points per game?
# lets define breakout as 30+ points
BREAKOUT_CUT = 30
#   league discussion has been on points for (as an aside, how about opp. median score?)
# ADD STATS STUFF TO THIS FUNCTION BUT KEEP THIS UP TO DATE
STATS_FIELDS = ["Weeks", "Weekly Points For", "Weekly Points Against", "Points For", "Points Against", "Points per Game",
                "Median Points Scored", "Points Against per Game", "Median Points Against", "3 game rolling average",
                "5 game rolling average", "Breakout Games", "Breakout Games Against", "Game Margin", "Blowout Index"]

# blowout index: closer to 0 is close games, bigger numbers indicate larger losses and wins

#   TODO: edge case for less than 3 or 5 weeks?


def team_stats(games):
    """calculate team stats. returns a dictionary of team ids with associated
       lists that define team stats. list has order:
       [total weeks, weekly points for, weekly points against, total points for, total points against, ppg, 
       median points scored, papg, median points against, 3 game rolling avg. points scored, 
       5 game rolling avg. points scored, breakouts for, breakouts against, game margin, blowout index]
    """
    weeks = len(games.keys())
    final_stats = {}
    weekly_pf = {}
    weekly_pa = {}
    weekly_bf = {}
    weekly_ba = {}

    for user in NAME_PAIRINGS:
        final_stats[user] = []
        weekly_pf[user] = []
        weekly_pa[user] = []
        weekly_bf[user] = []
        weekly_ba[user] = []

    for week in games:
        for matchup in games[week]:
            teams = list(games[week][matchup].keys())
            team0 = games[week][matchup][teams[0]]
            team1 = games[week][matchup][teams[1]]
            # use team data to fill in pf and opponent data for pa
            weekly_pf[teams[0]].append(team0["points"])
            weekly_pf[teams[1]].append(team1["points"])
            weekly_pa[teams[0]].append(team1["points"])
            weekly_pa[teams[1]].append(team0["points"])
            weekly_bf[teams[0]].append({})
            weekly_bf[teams[1]].append({})
            weekly_ba[teams[0]].append({})
            weekly_ba[teams[1]].append({})
            for player0, player1 in zip(team0["players_points"].keys(), team1["players_points"].keys()):
                if team0["players_points"][player0] >= BREAKOUT_CUT:
                    weekly_bf[teams[0]][week -
                                        1][player0] = team0["players_points"][player0]
                    weekly_ba[teams[1]][week -
                                        1][player0] = team0["players_points"][player0]
                elif team1["players_points"][player1] >= BREAKOUT_CUT:
                    weekly_ba[teams[0]][week -
                                        1][player1] = team1["players_points"][player1]
                    weekly_bf[teams[1]][week -
                                        1][player1] = team1["players_points"][player1]

    for user in NAME_PAIRINGS:
        final_stats[user].append(weeks)
        final_stats[user].append(weekly_pf[user])
        final_stats[user].append(weekly_pa[user])
        pf = sum(weekly_pf[user])
        pa = sum(weekly_pa[user])
        final_stats[user].append(round(pf, 2))
        final_stats[user].append(round(pa, 2))
        final_stats[user].append(round(pf/weeks, 2))
        final_stats[user].append(round(statistics.median(weekly_pf[user]), 2))
        final_stats[user].append(round(pa/weeks, 2))
        final_stats[user].append(round(statistics.median(weekly_pa[user]), 2))

        rolling_3 = []
        for i in range(weeks):
            if i < 3:
                rolling_3.append(round(sum(weekly_pf[user][0:i+1])/(i+1), 2))
            else:
                rolling_3.append(round(sum(weekly_pf[user][i-2:i+1])/3, 2))
        final_stats[user].append(rolling_3)

        rolling_5 = []
        for i in range(weeks):
            if i < 5:
                rolling_5.append(round(sum(weekly_pf[user][0:i+1])/(i+1), 2))
            else:
                rolling_5.append(round(sum(weekly_pf[user][i-4:i+1])/5, 2))
        final_stats[user].append(rolling_5)

        final_stats[user].append([len(k.keys()) for k in weekly_bf[user]])
        final_stats[user].append([len(k.keys()) for k in weekly_ba[user]])
        final_stats[user].append(
            [round(weekly_pf[user][week] - weekly_pa[user][week], 2) for week in range(weeks)])
        final_stats[user].append(round(sum(
            [final_stats[user][-1][i]**2 for i in range(weeks)]) / weeks, 2))  # mean squared margin

        # ADD HERE

    return final_stats


stats = team_stats(all_games)

In [10]:
# sanity check the weeks and uncomment here if you wanna

# for week in all_games:
#     print(f"------------Week {week}------------")
#     for matchup in all_games[week]:
#         for team in all_games[week][matchup]:
#             print(f" {NAME_PAIRINGS[str(team)]}: {all_games[week][matchup][team]['points']}")
#         print()

------------Week 1------------
 My Goat Loves You: 171.86
 RB sanctuary: 113.42

 The Brutherhood: 116.1
 Thicc King: 127.06

 tealeaves: 170.38
 Tree Leaves: 83.02

 the noés: 105.08
 AssNTitties: 126.38

 SideThicc #2: 84.62
 billcap: 126.5

------------Week 2------------
 My Goat Loves You: 127.6
 billcap: 108.56

 The Brutherhood: 129.16
 AssNTitties: 166.7

 tealeaves: 94.42
 SideThicc #2: 131.7

 Tree Leaves: 131.82
 Thicc King: 138.72

 the noés: 120.32
 RB sanctuary: 97.48

------------Week 3------------
 My Goat Loves You: 108.52
 AssNTitties: 152.14

 The Brutherhood: 109.72
 tealeaves: 81.5

 Tree Leaves: 150.24
 SideThicc #2: 118.44

 the noés: 140.54
 Thicc King: 133.22

 billcap: 112.28
 RB sanctuary: 84.62

------------Week 4------------
 My Goat Loves You: 109.8
 tealeaves: 159.12

 The Brutherhood: 108.42
 Tree Leaves: 124.98

 the noés: 123.32
 billcap: 118.14

 SideThicc #2: 117.62
 Thicc King: 117.36

 AssNTitties: 138.68
 RB sanctuary: 138.02

------------Week 5---

## League Status

Hello boys, I enjoyed last week so I will be doing some more analysis, this time with regards to roster construction. As a reminder, league standings are currently as follows:

```
Noe's Loyal Lighting Salesman Division
1. Nahome (11-5)
2. Ben (8-8)
3. Noe (8-8)
4. Will (7-9)
5. Stephane (3-13)

Noah's SideThiccs
1. Noah (11-5)
2. Praveen (11-5)
3. Stian (9-7)
4. Tom (7-9)
5. Tegran (5-11)
```

This weeks power rankings that I released on Tuesday had the most movement in weeks,  as Nahome fell to third and Stephane's trade paid off in the week's second highest scoring performance.

```
1. Noah
2. Praveen
3. Nahome
4. Ben
5. Stian
6. Noe
7. Tom
8. Will
9. Stephane
10. Tegran
```

If the playoffs started today, we would have ```Praven vs. Noe``` and ```Stian vs. Ben``` in the opening rounds, with Noah and Nahome both earning byes. In the toilet bowl, we would have ```Tegran vs. Will``` and ```Stephane vs. Tom```.

## Weekly Focus

For the week 8 deep dive, I will be creating and digging into a stat that Tom inspired me to try as a joke. Tom asked to do a wins above replacement stat (WAR) for Aiyuk, who obviously has not been the best fantasy performer this year.  Diligent readers probably noticed I wrote 'Aiyuk is ASS' as his WAR value. Regardless, I wanted to do something similar for this week since it will be pretty interesting to see the degree to which a player impacts a team. In fantasy, some position groups are simply more important because the dropoffs in player output is so much more drastic (This is why we generally draft RBs before WRs before QBs!). Since we have less games to sample from, I will try to analyze points above replacement (PAR) for players on a roster. Hopefully this will give some insight into the top performers on a roster and their overall contributions to a players' team. 

## Applying Rigor to PAR (and PAA)
#### This section dives more deeply into the logic behind my calculations and motivation for creating two classes of statistic. I would recommend reading it, but if that doesn't interest you as much you can skip it

The definition of WAR as stated on the Baseball-Reference website is “A single number that presents the number of wins the player added to the team above what a replacement player . . . would add.” For their purposes, a replacement player is defined as a player taken from the top minor league level. Of course, we don't have this selection luxury in fantasy which begs the question of what our replacement calculation should use. There is also debate in the baseball community, which I think informs our decision making process. Just due to the nature of rosters and player ownership, an average player may actually be difficult to acquire and require a team to trade value in pursuing their acquisition. With that said, it is clear that a true 'replacement' level player is one that is able to be picked up just off the waiver wire. In this sense, most players we start on a weekly basis have a positive points above replacement statistic. This implies that our aggregate team PAR score might be judging our teams relative to one we can create off the waiver wire.

This then suggests that the average rostered player would give us a better sense of the amount a team is helped by an individual **relative** to other teams. Teams are generally not starting replacement players on a weekly basis, so does it really make sense to peg point production to players that are not started? Because of this fact, I came to the solution of binning players into tiers and comparing their point production to these tiers. In our league, we start 1 QB, 2 RBs, 2 WRs, 1 TE, and 2 FLEX players (as well as 1 team defense and 1 kicker) and have 10 total teams. If we can calculate what a player in a given starting spot is expected to score, we could analyze how many points they contribute relative to a starting lineup consisting solely of bang average players. Let's drill down into running backs as a concrete example of this methodology. If in our league the top 10 running backs in a week scored 20, 19, 18, 17, 16, 15, 14, 13, 12, and 11 points respectively, we might say that the expected point return of eachs team's top running back (their RB1) was 14.5 points. In this respect, we could infer that a replacement (or perhaps more specifically an average) RB1 would be worth 14.5 points to their team. Thus the top scoring RB would have added 5.5 points to their team. We will call this value the Points Added Over Average (PAA)

We could even go further and note that a player likely has a replacement for a given player sitting on their bench. In this sense, a players' value to a given team is actually a function of the bench strength for this team. Whereas the previous two measures are trying to analyze a player relative to the waiver wire and other teams respectively, this calculation would analyze a players' value relative to a teams' own players. If a team has good RBs or a strong backup quarterback, a given starter may not make as big of a difference to the team compared to if they were rostered by another user. Because of this team based approach, we will call this measure Points Added Over Substitution (PAS).

I believe all these statistics have uses. We saw just this week that Derrick Henry was hurt and Noah was forced to replace him off the waiver wire. We may expect his points to drop by the value of Derrick Henry's PAR score. We all know that Stephane has had a brutal start to his season. In his case, it may be more interesting to compare how the running backs he drafted are performing relative to the top RB options on other squads. For this case we can analyze the PAA stat of his starting RBs to infer the amount they have hurt his season compared to other positional players. For PAS, Nahome has an extremely strong roster of WRs, so the marginal value of his starting WRs to the point total that his team scores is lower than if those WRs were placed on a team lacking at the position. I'll do my best to provide an overview of these analyses for each team, and then provide a league wide overview of team/position strength.

The presence of bye weeks presents a slight difficulty in these calculations. I am making the conscious decision to ignore bye weeks even though some players have not yet had them while others have (and thus have had an extra week of production). My logic is for injured players, I would not want to remove weeks for when they are injured from this calculation since they would have scored 0 points if you had started them. Since each player will have a bye week at some point this season, I think the logic for leaving bye week production as 0 points is even more sound than for including injury weeks in the calculation. Essentially the argument boils down to this: If you are relying on a player to be your RB1 and they score 0 points on their bye week, other teams are still starting players who are RB1s. I believe this difference in points scored should therefore still count towards a players' production output.

## Generating Player PAR Estimates

Generating PAR estimates will generally be simpler than PAA estimates. I will just use the average points scored by players on the waiver wire (who have scored a non-zero amount of points, under the assumption that this is the true pool of replacement players) to generate an estimate of the average score for a replacement player at a given position. I will then compare this to the actual points scored by players on active rosters to generate the difference in points scored by a given player vs. this replacement pool. This stat can then be aggregated for a given team or for a position subgroup on a team to calculate the team and positional strength of a team relative to a team made up solely of replacement level players.

In [13]:
# lets grab all the players Sleeper has, lets only run this once cause the api says it does not need to be called more than once per day
players = sleeper_players.get_all_players()

In [32]:
# unfortunately, this just gives us the player metadata and not their stats
# the api wrapper im using doesn't enable us to grab stats directly, so we
# will do it ourselves with the requests package and the 
# api route: https://api.sleeper.app/stats/nfl/player/{XXXX}?season_type=regular&season=2021&grouping=season
# what's nice is that if a player has no stats, they just return a null response so we can populate our
# stats dictionary with only players that have a point very easily

# also only run this once lol it takes like 45 minutes

PLAYER_ROUTE = "https://api.sleeper.app/stats/nfl/player/"

# literally gonna comment this out so it doesn't accidentally run
# player_stats = {}

# for num, player in enumerate(players):
#     response = requests.get(url=PLAYER_ROUTE+player, params={"season_type": "regular", "season": "2021", "grouping": "season"})
#     if response.json():
#         player_stats[player] = response.json()["stats"]
#     if num % 500 == 0:
#         print(f"Done with player {num} out of  {len(players.keys())}")

Done with player 0 out of  8094
Done with player 500 out of  8094
Done with player 1000 out of  8094
Done with player 1500 out of  8094
Done with player 2000 out of  8094
Done with player 2500 out of  8094
Done with player 3000 out of  8094
Done with player 3500 out of  8094
Done with player 4000 out of  8094
Done with player 4500 out of  8094
Done with player 5000 out of  8094
Done with player 5500 out of  8094
Done with player 6000 out of  8094
Done with player 6500 out of  8094
Done with player 7000 out of  8094
Done with player 7500 out of  8094
Done with player 8000 out of  8094


In [90]:
# since we already know from our api requests that players which return a response have stats, we just need 
# to filter on the pts_half_ppr field (which individual defensive players do not have)
scoring_offensive_players = {}

for player in player_stats:
    if "pts_half_ppr" in player_stats[player] and players[player]["position"] in FANTASY_POSITIONS:
        try:
            scoring_offensive_players[player] = {"points": player_stats[player]["pts_half_ppr"]}
            scoring_offensive_players[player]["fantasy_positions"] = players[player]["fantasy_positions"]
            scoring_offensive_players[player]["position"] = players[player]["position"]
            scoring_offensive_players[player]["name"] = players[player]["full_name"]
        except:
            scoring_offensive_players[player] = {"points": player_stats[player]["pts_half_ppr"]}
            scoring_offensive_players[player]["fantasy_positions"] = players[player]["fantasy_positions"]
            scoring_offensive_players[player]["position"] = players[player]["position"]
            scoring_offensive_players[player]["name"] = players[player]["player_id"]

In [83]:
rostered_players = {}

for roster in rosters:
    team = roster["players"]
    for player in team:
        # sometimes players on IR will have 0 points scored, so we can build their record if
        # they are rostered
        try:
            rostered_players = {**rostered_players, **{player: scoring_offensive_players[player]}}
        except KeyError:
            rostered_players = {**rostered_players, **{player: {"points": 0, 
                                                       "name": players[player]["full_name"], 
                                                       "position": players[player]["position"], 
                                                       "fantasy_positions": players[player]["fantasy_positions"]}}
                                                       }
to_remove = []
for player in scoring_offensive_players:
    if player in rostered_players:
        to_remove.append(player)

replacement_players = {key: scoring_offensive_players[key] for key in scoring_offensive_players if key not in rostered_players}

In [98]:
# we should now calculate the replacement average points for each position
# if limit is set to None, there is no limit on the replacement players and
# all are used to calculate the average. It may be more realistic to just use
# the top # = LIMIT players on the waiver wire at a position, since those are
# the replacement options you would actually pickup
replacement_averages = {}
LIMIT = 10

for position in FANTASY_POSITIONS:
    pos_sum = 0
    pos_players = 0 if not LIMIT else LIMIT
    top_players = []

    for player in replacement_players:
        # Flex encompasses three positional replacement possibilities, so we
        # must take all three into account
        if position == "FLEX":
            for subposition in ["RB", "WR", "TE"]:
                if subposition in replacement_players[player]["fantasy_positions"]:
                    if not LIMIT:
                        pos_sum += replacement_players[player]["points"]
                        pos_players += 1
                    else:
                        if len(top_players) < LIMIT:
                            top_players.append(
                                replacement_players[player]["points"])
                        else:
                            p_points = replacement_players[player]["points"]
                            top_players = [p_points if p_points > min(
                                top_players) else curr_top for curr_top in top_players]
                        pos_sum = sum(top_players)
        else:
            if position in replacement_players[player]["fantasy_positions"]:
                if not LIMIT:
                    pos_sum += replacement_players[player]["points"]
                    pos_players += 1
                else:
                    if len(top_players) < LIMIT:
                        top_players.append(
                            replacement_players[player]["points"])
                    else:
                        p_points = replacement_players[player]["points"]
                        top_players = [p_points if p_points > min(top_players) else curr_top for curr_top in top_players]
                    pos_sum = sum(top_players)

    
    replacement_averages[position] = pos_sum / pos_players

replacement_averages


{'QB': 133.16000000000003,
 'RB': 57.60000000000001,
 'WR': 72.5,
 'TE': 49.199999999999996,
 'FLEX': 72.5,
 'DEF': 77.0,
 'K': 70.0}

In [100]:
# After we have the expected replacement score, we can simply calculate the difference
# between this average score and a players' total score to find their points above replacement.

for roster in rosters:
    roster["PAR_values"] = []
    for player in roster["players"]:
        player_position = rostered_players[player]["position"]
        player_points = rostered_players[player]["points"]
        roster["PAR_values"].append(player_points - replacement_averages[player_position])


for player, par_value in zip(rosters[0]["players"], rosters[0]["PAR_values"]):
    print(f"{rostered_players[player]['name']} has scored {rostered_players[player]['points']} pts. This is {round(par_value, 2)} points {'above' if par_value > 0 else 'below'} a typical replacement {rostered_players[player]['position']}")


Greg Zuerlein has scored 62.0 pts. This is -8.0 points below a typical replacement K
Travis Kelce has scored 101.5 pts. This is 52.3 points above a typical replacement TE
Christian McCaffrey has scored 50.4 pts. This is -7.2 points below a typical replacement RB
Matthew Stafford has scored 186.48 pts. This is 53.32 points above a typical replacement QB
Tim Patrick has scored 75.4 pts. This is 2.9 points above a typical replacement WR
T.J. Hockenson has scored 82.8 pts. This is 33.6 points above a typical replacement TE
A.J. Brown has scored 86.7 pts. This is 14.2 points above a typical replacement WR
Myles Gaskin has scored 76.9 pts. This is 19.3 points above a typical replacement RB
Jerry Jeudy has scored 16.1 pts. This is -56.4 points below a typical replacement WR
CeeDee Lamb has scored 106.2 pts. This is 33.7 points above a typical replacement WR
Laviska Shenault has scored 48.7 pts. This is -23.8 points below a typical replacement WR
Clyde Edwards-Helaire has scored 46.5 pts. This